In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [4]:
# Carregar os dados - Regressão
df = pd.read_csv("datasets/colesterol.csv")

In [5]:
# Visualizar estrutura dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [6]:
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33,85.1,186,199.63
1,2,A,Não,Moderado,68,105.0,184,236.98
2,3,O,Não,Alto,25,64.8,180,161.79
3,4,A,Não,Alto,43,120.2,167,336.24
4,5,AB,Não,Baixo,79,88.5,175,226.23
5,6,B,Não,Baixo,68,66.8,170,185.31
6,7,A,Sim,Baixo,60,117.3,181,289.33
7,8,O,Sim,Moderado,35,86.9,174,216.48
8,9,O,Não,Baixo,62,81.3,166,235.30
9,10,B,Sim,Alto,44,32.7,165,97.79


In [7]:
# Ajustar Dataframe
df.drop(columns=["Id"], axis=1, inplace=True)

# Aplicar One Hot Encoding nas variaveis categóricas
df = pd.get_dummies(df, columns=["Grupo Sanguíneo", "Fumante", "Nível de Atividade"])

In [8]:
df.head(10)

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,0,0,1,0,0,1,0,1,0
1,68,105.0,184,236.98,1,0,0,0,1,0,0,0,1
2,25,64.8,180,161.79,0,0,0,1,1,0,1,0,0
3,43,120.2,167,336.24,1,0,0,0,1,0,1,0,0
4,79,88.5,175,226.23,0,1,0,0,1,0,0,1,0
5,68,66.8,170,185.31,0,0,1,0,1,0,0,1,0
6,60,117.3,181,289.33,1,0,0,0,0,1,0,1,0
7,35,86.9,174,216.48,0,0,0,1,0,1,0,0,1
8,62,81.3,166,235.30,0,0,0,1,1,0,0,1,0
9,44,32.7,165,97.79,0,0,1,0,0,1,1,0,0


### Treinar modelo de Regressão com Lasso (L1)

In [9]:
# Separar X e y
X = df.drop(columns=["Colesterol"], axis=1)
y = df["Colesterol"]

In [10]:
# Separar Treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [11]:
# Treinar o modelo de regressão linear múltipla com Lasso
# Quanto maior o alpha, maior a penalização e mais coeficientes tendem a ser reduzidos a zero
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [12]:
# Mostrar importância de Features
def importancia_features(modelo):
    importance = np.abs(modelo.coef_)
    print("Importância das Features:")
    for i, feature in enumerate(modelo.feature_names_in_):
        print(f"{feature}: {importance[i]}")


In [13]:
# Importancia das Features - Lasso
importancia_features(model_lasso)

Importância das Features:
Idade: 0.017349176463551177
Peso: 2.4911793204672508
Altura: 2.20863191913672
Grupo Sanguíneo_A: 0.05063677331183119
Grupo Sanguíneo_AB: 1.032901896958969
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 1.6422787796520806
Fumante_Sim: 0.0
Nível de Atividade_Alto: 1.485309859678915
Nível de Atividade_Baixo: 0.0
Nível de Atividade_Moderado: 0.17448169861080404


In [14]:
# Função para avaliar performance
def performance_regressao(modelo, X_test, y_test):
    # Faz predição com o modelo no conjunto de teste
    y_pred = modelo.predict(X_test)
    # Avaliar desempenho
    mse = mean_squared_error(y_test, y_pred, squared=False)
    return mse

In [15]:
# Performance Regressão com Lasso
performance_regressao(model_lasso, X_test, y_test)

/home/vitor-linux/.local/share/virtualenvs/Topicos_Complementares-PXZMFool/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.96623471437868

In [20]:
model_lasso_cv = LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)
model_lasso_cv.fit(X, y)

LassoCV(alphas=[0.1, 0.5, 1], cv=5, random_state=51)

In [21]:
# Importancia das Features - LassoCV
importancia_features(model_lasso_cv)

Importância das Features:
Idade: 0.01754486106573828
Peso: 2.4847665015936133
Altura: 2.199233004768637
Grupo Sanguíneo_A: 0.0
Grupo Sanguíneo_AB: 1.7412230178317802
Grupo Sanguíneo_B: 0.0
Grupo Sanguíneo_O: 0.0
Fumante_Não: 1.8358701056169333
Fumante_Sim: 0.0
Nível de Atividade_Alto: 0.6961424797287861
Nível de Atividade_Baixo: 0.17642514217590116
Nível de Atividade_Moderado: 0.0


In [22]:
# Performance Regressão com LassoCV
performance_regressao(model_lasso_cv, X_test, y_test)

/home/vitor-linux/.local/share/virtualenvs/Topicos_Complementares-PXZMFool/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.872879747160336